In [1]:
import torch

In [5]:
from transformers import AutoModelForSeq2SeqLM, NllbTokenizerFast
model_name="facebook/nllb-200-3.3B"
# Initialize the tokenizer and model
tokenizer = NllbTokenizerFast.from_pretrained(model_name,tgt_lang="heb_Hebr",src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
#model.to('cuda')

In [7]:
@torch.no_grad
def translate_text(text):
    # Tokenize and translate the text
    encoded_text = tokenizer(text, return_tensors="pt")
    #manual fix to hf bug 
    encoded_text['input_ids'][:,1]=tokenizer.lang_code_to_id[tokenizer.src_lang]

    encoded_text={k:v.to(model.device) for k,v in encoded_text.items()}
    generated_tokens = model.generate(**encoded_text,forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang]).cpu()

    # Decode and return the translated text
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


text_to_translate = "Hello, how are you?"

translated_text = translate_text(text_to_translate)
print(translated_text)


מה שלומך?


In [9]:
model

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 2048, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 2048, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerNorm

In [8]:
translate_text(""" What were the specific allegations made by Avishai Greenzayg, Che
n Maanit, and Globes Publisher regarding the request to film, record, and broadc
ast the criminal proceedings against Benjamin Netanyahu before the Jerusalem Dis
trict Court, and what was the decision of the Supreme Court regarding this reque
st?""")

"מה היו הטענות הספציפיות של אבישי גרינזאג, צ'אן מאניט, ופרסום גלובס בנוגע לבקשת צילום, הקלטה והפצת ההליכים הפליליים נגד בנימין נתניהו בפני בית המשפט המחוזי בירושלים, ומה הייתה החלטת בית המשפט העליון בנוגע לבקשת זו?"

In [13]:
tokenizer.decode(tokenizer('')['input_ids'])

'eng_Latn</s>'

In [15]:
tokenizer.decode(tokenizer.encode(''))

'eng_Latn</s>'

In [50]:
translate_text("""What specific issues did the company "Tzofen Avtaha Shmira V'niki
yon" raise regarding the regulations for obtaining a license for security servic
es, and how did the court rule on their petition?""")

'האם החברה "צ\'ופן אבטחה שמירה וניקיה יון" העלתה סוגיות ספציפיות בנוגע לתקנות קבלת רישיון לשירותי אבטחה, ואיך פסק בית המשפט על עתירתם?'

In [51]:
translate_text("""What are the specific deadlines for submitting the written summar
ies of claims and responses, and the oral argument in the case involving "Klal I
nsurance Company" and the "Manager of Large Facilities"?""")

'האם ישנן מועדים ספציפיים להגיש את סיכום התביעות וההתגובות בכתב, ואת הטיעון הפנימי בתיק המעורב ב"חברת הביטוח קלל" ו"מנהל המתקנים הגדולים"?'

In [35]:
tokenizer.src_lang

'eng_Latn'

In [38]:
tokenizer.tgt_lang

'heb_Hebr'

In [36]:
tokenizer.lang_code_to_id[tokenizer.src_lang]

256047

In [32]:
tokenizer.decode(256184)

'?'

In [22]:
tokenizer.src_lang

'eng_Latn'

In [23]:
tokenizer._start_token_id

AttributeError: 'NllbTokenizerFast' object has no attribute '_start_token_id'